# Packages

In [1]:
import numpy as np 
import pandas as pd
import os
import time
import multiprocessing 
from joblib import Parallel, delayed
import contextlib
import joblib
import cv2 as cv
from scipy import signal
from sklearn.neighbors import BallTree
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import community as community_louvain
import glob
from pathlib import PurePath
from skimage.util import map_array
from PIL import Image
import matplotlib.pyplot as plt
import numba
import random
import seaborn as sns
from scipy.spatial.distance import cdist
import math

%load_ext watermark

%watermark
%watermark --iversions

Last updated: 2024-09-25T09:50:11.437749+02:00

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.10.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
CPU cores   : 16
Architecture: 64bit

seaborn   : 0.13.2
numpy     : 1.23.5
cv2       : 4.10.0
PIL       : 9.4.0
sklearn   : 1.3.0
joblib    : 1.4.2
scipy     : 1.10.0
networkx  : 3.3
community : 0.16
pandas    : 1.5.3
skimage   : 0.19.3
numba     : 0.56.4
matplotlib: 3.7.0



# Functions to compute neighborhood parameters

In [ ]:
def compute_neighbors_within_radius(X: list[float], 
                                    Y: list[float], 
                                    radius: float
                                    ) -> list[np.ndarray]:
    
    """
    Computes the neighbors of each point within a specified radius using BallTree for fast spatial queries.
    
    Parameters
    ----------
    X : array-like, shape (n_samples,)
        Array containing the x-coordinates of the points.
        
    Y : array-like, shape (n_samples,)
        Array containing the y-coordinates of the points.
        
    radius : float
        The radius within which to search for neighboring points around each point.
        
    Returns
    -------
    query : list of np.ndarray
        A list where each element is an array containing the indices of the neighbors for each point.
        Each array corresponds to the indices of points that are within the specified radius of the point.
    """
    
    points = np.asarray([X,Y]).transpose()
    tree = BallTree(points, leaf_size=10)    
    # output list of neighbors index for each point
    query = tree.query_radius(points, r=radius, count_only=False)   #index correspond to id_roi+1
    return query

def compute_partition(X: list[float], 
                      Y: list[float], 
                      query: list[np.ndarray]
                      ) -> dict[int, int]:

    """
    Computes the partition of points based on a graph constructed from neighbor relationships.
    The Louvain method is used to detect communities in the graph.

    Parameters
    ----------
    X : list of float
        List containing the x-coordinates of the points.

    Y : list of float
        List containing the y-coordinates of the points.

    query : list of np.ndarray
        List where each element is an array containing the indices of neighbors for each point.

    Returns
    -------
    partition : dict
        A dictionary where keys are node indices and values are the community assignments (partition label).
        Each node is assigned to a community detected by the Louvain algorithm.
    """

    G = nx.Graph()
    pos = {i: (x, y) for i, (x, y) in enumerate(zip(X,Y))}
    G.add_nodes_from(pos.keys())
    
    for i in range(len(query)): 
        for j in query[i]:
            if i != j:
                G.add_edge(i, j)  
    
    partition = community_louvain.best_partition(G)    
    
    return partition


def compute_local_order(df: pd.DataFrame, 
                        query: list[np.ndarray], 
                        idx: int
                        ) -> float:

    """
    Computes the local nematic order for a given point by comparing its orientation with its neighbors.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the orientation data for each point. Must have columns 'x_orientation' and 'y_orientation'
        representing the x and y components of the orientation vector for each point.

    query : list of np.ndarray
        List where each element is an array containing the indices of neighbors for each point.

    idx : int
        The index of the point for which to compute the local order.

    Returns
    -------
    nematic_order : float
        The local nematic order, a value between 0 and 1, indicating the degree of alignment of the point's orientation
        with its neighbors. A value of 0 indicates no alignment, while 1 indicates perfect alignment.
    """

    neighbors = query[idx]
    
    vector_ref = [0,1]
    unit_vector_ref = vector_ref / np.linalg.norm(vector_ref)
    
    if len(neighbors) > 1:
        
        neighbors_orientation = []
        
        for neighbor in neighbors:
            
            vector = [df['x_orientation'][neighbor], df['y_orientation'][neighbor]]
            unit_vector = vector / np.linalg.norm(vector)
            dot_product = np.dot(unit_vector_ref, unit_vector)
            angle = np.arccos(dot_product)
            neighbors_orientation.append(angle)

        orientation_array = np.asarray(neighbors_orientation)
        nematic_order = np.sqrt(np.mean(np.cos(2*orientation_array))**2 + np.mean(np.sin(2*orientation_array))**2)
        if nematic_order==np.nan:
            print(nematic_order)
        
    else:
        nematic_order = 0
        
    return nematic_order

def compute_closest_distance(X: list[float],
                             Y: list[float]
                             ) -> tuple[list[float], list[int]]:
    """
    Computes the closest distance for each point and returns the distances and corresponding neighbor indices.

    Parameters
    ----------
    X : list of float
        List containing the x-coordinates of the points.

    Y : list of float
        List containing the y-coordinates of the points.

    Returns
    -------
    distances : list of float
        A list containing the distance to the nearest neighbor for each point.

    indices : list of int
        A list containing the index of the nearest neighbor for each point.
    """

    pts=np.array([[x,y] for x,y in zip(X,Y)])
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(pts)
    distances, indices = nbrs.kneighbors(pts)
    
    return list(distances[:,1]), list(indices[:,1])

@numba.jit(nopython=True)
def compute_radial_density(points: np.ndarray, 
                           radius: float
                           ) -> np.ndarray:
    """
    Computes the radial density for each point, defined as the number of points within a specified radius
    divided by the area of the circle formed by the radius.

    Parameters
    ----------
    points : np.ndarray, shape (n_points, 2)
        A 2D array where each row represents the x and y coordinates of a point.

    radius : float
        The radius within which to count neighboring points.

    Returns
    -------
    densities : np.ndarray, shape (n_points,)
        An array containing the density for each point. The density is computed as the number of points
        within the given radius divided by the area of the circle formed by the radius.
    """
    n_points = points.shape[0]
    densities = np.zeros(n_points)
    for i in range(n_points):
        count_within_radius = 0
        for j in range(n_points):
            if i != j:
                distance = np.sqrt((points[i, 0] - points[j, 0]) ** 2 + (points[i, 1] - points[j, 1]) ** 2)
                if distance <= radius:
                    count_within_radius += 1
        area = np.pi * radius ** 2
        densities[i] = count_within_radius / area
    return densities


def compute_min_distance_to_cell_population(df: pd.DataFrame, points: np.ndarray) -> np.ndarray:

    """
    Computes the minimum distance from each point (representing cell centroids) in a DataFrame to a given set of points.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing at least two columns, 'x_centroid' and 'y_centroid', which represent the x and y
        coordinates of cell centroids.
        
    points : np.ndarray, shape (n_points, 2)
        A 2D array where each row represents the x and y coordinates of a point to which distances will be computed.

    Returns
    -------
    min_distances : np.ndarray, shape (n_cells,)
        An array containing the minimum distance from each cell centroid to the nearest point in the `points` array.
    """
    
    centroids = df[['x_centroid', 'y_centroid']].to_numpy()
    distances = cdist(centroids, points)
    min_distances = distances.min(axis=1)
    
    return min_distances


def compute_order(df: pd.DataFrame, 
                  x: list[float], 
                  y: list[float], 
                  radius: float
                  ) -> list[float]:
    
    """
    Computes the local order parameter for each point in the DataFrame based on neighboring points within a specified radius.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing orientation data for each point. The DataFrame must include 'x_orientation' and 'y_orientation'
        columns to compute local order.
        
    x : list of float
        List of x-coordinates for the points.
    
    y : list of float
        List of y-coordinates for the points.
    
    radius : float
        The radius within which to consider neighboring points for the local order computation.

    Returns
    -------
    orders : list of float
        A list containing the local order values for each point, based on the orientations of its neighbors within the given radius.
    """

    query = compute_neighbors_within_radius(x, y, radius=radius)
    orders = []
    for idx in df.index:
        orders.append(compute_local_order(df, query, idx))

    return orders


def compute_neighborhood_features(path: str, 
                                  out_dir: str,
                                  density_radius: int = 40,
                                  order_radius: int = 20,
                                  community_radius: int = 80
                                  ) -> pd.DataFrame:
    
    """
    Computes neighborhood features for tumor and stroma compartments and saves them to a new HDF5 file. The features
    include radial density, local order, community (subunit) partitioning, closest neighbor information, and distances
    between compartments.

    Parameters
    ----------
    path : str
        Path to the input HDF5 file containing the feature data for tumor and stroma compartments.
    
    out_dir : str
        Directory where the processed neighborhood feature file will be saved.

    density_radius : int, optional, default=40
        The radius (in arbitrary units) used to compute the radial density of tumor and stroma cells.

    order_radius : int, optional, default=20
        The radius (in arbitrary units) used to compute the local order of tumor and stroma cells.

    community_radius : int, optional, default=80
        The radius (in arbitrary units) used to compute the community (subunit) partitioning of tumor and stroma cells.

    Returns
    -------
    df_feat : pd.DataFrame
        DataFrame containing the updated feature set with neighborhood properties for both tumor and stroma compartments.
    """

    df_feat = pd.read_hdf(path, 'df')
    out_name = PurePath(path).name.replace('features_tumor_stroma', 'features_neighborhood')
    out_path = os.path.join(out_dir, out_name)

    out_paths = glob.glob(out_dir + "/*.h5") 

    if out_path not in out_paths:

        df_tumor = df_feat.query("compartment == 'tumor'").reset_index(drop=True)
        df_stroma = df_feat.query("compartment == 'stroma'").reset_index(drop=True)

        x_tumor, y_tumor = df_tumor['x_centroid'].values, df_tumor['y_centroid'].values
        x_stroma, y_stroma = df_stroma['x_centroid'].values, df_stroma['y_centroid'].values

        tumor_points = np.array([[x,y] for x,y in zip(x_tumor,y_tumor)])
        stroma_points = np.array([[x,y] for x,y in zip(x_stroma,y_stroma)])

        # Tumor 
        query = compute_neighbors_within_radius(x_tumor, y_tumor, radius=community_radius)
        partition = compute_partition(x_tumor, y_tumor, query)
        df_tumor['subunit_id'] = partition

        distances, indices = compute_closest_distance(x_tumor, y_tumor)
        df_tumor['closest_neighbor_id'] = indices
        df_tumor['closest_distance'] = distances

        df_tumor['radial_density'] = compute_radial_density(tumor_points, radius=density_radius)
        df_tumor['order'] = compute_order(df_tumor, x_tumor, y_tumor, radius=order_radius)
        
        df_tumor['dist_to_stroma'] = compute_min_distance_to_cell_population(df_tumor, stroma_points)

        # Stroma
        query = compute_neighbors_within_radius(x_stroma, y_stroma, radius=community_radius)
        partition = compute_partition(x_stroma, y_stroma, query)
        df_stroma['subunit_id'] = partition

        distances, indices = compute_closest_distance(x_stroma, y_stroma)
        df_stroma['closest_neighbor_id'] = indices
        df_stroma['closest_distance'] = distances

        df_stroma['radial_density'] = compute_radial_density(stroma_points, radius=density_radius)
        df_stroma['order'] = compute_order(df_stroma, x_stroma, y_stroma, radius=order_radius)
        
        df_stroma['dist_to_stroma'] = compute_min_distance_to_cell_population(df_stroma, tumor_points)

        # Concat tumor and stroma data and export

        df_feat = pd.concat([df_tumor, df_stroma]).reset_index(drop=True)
    
        df_feat.to_hdf(out_path, key='df', mode='w')

        return df_feat
        

# Batch processing

In [ ]:
FEAT_DIR = os.path.join('', 'features_tumor_stroma')
OUT_DIR = r''

feature_paths = glob.glob(FEAT_DIR + "/*.h5")

from joblib import Parallel, delayed

Parallel(n_jobs=10)(delayed(compute_neighborhood_features)(path) for path in feature_paths)

